Here is how to feed lua code into BizHawk from python script. Idealy there is no need to change the "action.lua" script. However, the rom_path variable needs to be changed to the path to your rom file.

In [ ]:
%pylab inline
import subprocess
import os

In [ ]:
FPS = 60
bizhawk_path = 'BizHawk-2.2.2/'
rom_path = '' #Path to the .smc rom file
preparation = False

proc = subprocess.Popen([bizhawk_path + 'EmuHawk.exe',
                         rom_path,
                         '--lua=../action.lua',],
                        stdout=subprocess.PIPE,
                        stdin=subprocess.PIPE)

while True:
    out_line = proc.stdout.readline()
    
    if out_line[:5] == b'start':
        #get rom name
        rom_info = out_line[6:-1].decode()
        print(rom_info)
        preparation = True
        
    if out_line == b'':
        break
        
    #start feeding code to lua script
    if preparation:
        #speed up
        proc.stdin.write(b'client.speedmode(400)')
        proc.stdin.flush()
        
        #advance 10 seconds
        for i in range(10 * FPS):
            proc.stdin.write(b'emu.frameadvance()')
            proc.stdin.flush()
        
        #save and show screenshot
        proc.stdin.write(b'client.screenshot("temp.png")')
        proc.stdin.write(b'io.stdout:write("saved")')
        proc.stdin.flush()
        while proc.stdout.readline() != b'saved\n':
            pass
        imshow(imread('temp.png'));
        
        
        #show memory
        proc.stdin.write(b'io.stdout:write(mainmemory.getcurrentmemorydomainsize())')
        proc.stdin.flush()
        ram_size = int(proc.stdout.readline()[:-1].decode('utf-8'))
        print('RAM size:', ram_size)
        
        #Show the first 8192 bytes
        ram = []
        for i in range(8192):
            proc.stdin.write(b'io.stdout:write(mainmemory.readbyte(' + str.encode(str(i)) + b'))')
            proc.stdin.flush()
            ram.append(int(proc.stdout.readline()[:-1].decode('utf-8')))
        ram = np.reshape(ram, (8192 // 256, 256))
        figure()
        imshow(ram);
        
        #save state to file
        proc.stdin.write(b'savestate.save("temp.state")')
        proc.stdin.flush()
        
        #press a button on the controller
        proc.stdin.write(b'buttons = {};buttons["Start"] = 0 joypad.set(buttons, 1)')
        proc.stdin.flush()
        
        #advance 10 more seconds
        for i in range(10 * FPS):
            proc.stdin.write(b'emu.frameadvance()')
            proc.stdin.flush()
        
        #show another screenshot
        proc.stdin.write(b'client.screenshot("temp.png")')
        proc.stdin.write(b'io.stdout:write("saved")')
        proc.stdin.flush()
        while proc.stdout.readline() != b'saved\n':
            pass
        figure()
        imshow(imread('temp.png'));
        
        #load state and show screenshot to confirm load
        proc.stdin.write(b'savestate.load("temp.state")')
        proc.stdin.write(b'client.screenshot("temp.png")')
        proc.stdin.write(b'io.stdout:write("saved")')
        proc.stdin.flush()
        while proc.stdout.readline() != b'saved\n':
            pass
        figure()
        imshow(imread('temp.png'));
        
        break
        
proc.terminate()
os.remove('temp.png')
os.remove('temp.state')